In [ ]:
import pandas as pd

In [ ]:
customers = pd.read_csv("data/customers.txt")
customers

In [ ]:
products = pd.read_csv("data/products.txt")
products = products.drop(columns=['dateOnSite'])
products

In [ ]:
train_data = pd.read_csv("data/labels_training.txt")
train_data

In [ ]:
# test_data = pd.read_csv("data/labels_predict.txt")
# test_data

In [ ]:
views = pd.read_csv("data/views.txt")
views = views.drop(columns=['imageZoom']) # discard imageZoom column since all 0 but 1 value
views

In [ ]:
# purchases = pd.read_csv("data/purchases.txt")
# purchases

In [ ]:
aggr_views = views.groupby(['customerId','productId']).sum() # aggregate the views of a customer of a product by summing
aggr_views

In [ ]:
# from the purchases dataframe, get the number of purchases per user (before which date ?), and number of purchases per product (before which date?)
# df['customerId'][[0,1,13481428]]

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from main import load_training_data, fill_missing_values

features, target, customerId = load_training_data()

features = fill_missing_values(features)
